Your Own Analysis 2 
GOAL
1) Lin28a CLIP-seq Binding 위치 주석 (CDS, UTR)



1. Lin28a CLIP-seq Binding 위치 주석 (CDS, UTR)

In [3]:
import os
os.chdir("/home/sohyunkum2/binfo/YOA/data")

!bedtools bamtobed -i CLIP-35L33G.bam > CLIP-35L33G.bed

In [26]:
# GTF 파일에서 CDS와 3'UTR 영역 추출
!awk '$3 == "CDS"' gencode.gtf > CDS.gtf
!awk '$3 =="UTR"' gencode.gtf > utr.gtf

In [27]:
# GTF -> BED 변환 (Intersect 하기 위함)
!gtf2bed < CDS.gtf > CDS.bed
!gtf2bed < utr.gtf > utr.bed

In [28]:
# Binding 위치와 CDS/3'UTR intersect 
!bedtools intersect -a CLIP-35L33G.bed -b CDS.bed -wa -wb > binding_in_CDS.bed
!bedtools intersect -a CLIP-35L33G.bed -b utr.bed -wa -wb > binding_in_UTR.bed

***** WARNING: File CLIP-35L33G.bed has inconsistent naming convention for record:
GL456210.1	7142	7157	SRR458758.6758016	40	-

***** WARNING: File CLIP-35L33G.bed has inconsistent naming convention for record:
GL456210.1	7142	7157	SRR458758.6758016	40	-

***** WARNING: File CLIP-35L33G.bed has inconsistent naming convention for record:
GL456210.1	7142	7157	SRR458758.6758016	40	-

***** WARNING: File CLIP-35L33G.bed has inconsistent naming convention for record:
GL456210.1	7142	7157	SRR458758.6758016	40	-



In [4]:
#!pip install seaborn

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re

#1. count 데이터 불러오기 
df = pd.read_csv("read-counts.txt", sep='\t', comment='#', index_col=0)


#2. mRNA abundance 변화 계산 
df["RNA_change"] = df["RNA-siLin28a.bam"] - df["RNA-siLuc.bam"] 

#3. binding_in_CDS.bed / UTR.bed에서 transcript_id 추출
cds = pd.read_csv("binding_in_CDS.bed", sep ="\t", header=None)
utr = pd.read_csv("binding_in_UTR.bed", sep = "\t", header=None)

def extract_tx(df):
    col = df[15].astype(str)
    return set(col.str.extract(r'gene_id "([^"]+)"')[0].dropna())

cds_tx = extract_tx(cds)
utr_tx = extract_tx(utr)

#4. gene별 binding 부위 주석 추가
def get_binding_region(tx):
    if tx in cds_tx and tx in utr_tx:
        return "both"
    elif tx in cds_tx:
        return "CDS"
    elif tx in utr_tx:
        return "UTR"
    else:
        return "none"
    
df["binding_region"] = df.index.map(get_binding_region)

In [8]:
print(df["binding_region"].value_counts())


binding_region
none    35446
both    17348
UTR      1526
CDS      1039
Name: count, dtype: int64


In [7]:
# RNA 관련 열만 추출
rna_cols = ["RNA-siLin28a.bam", "RNA-siLuc.bam", "RNA-control.bam", "CLIP-35L33G.bam", "RPF-siLin28a.bam", "RPF-siLuc.bam"]

# 두 열 모두 0인 gene 수 확인
zero_rna_genes = df[(df[rna_cols] == 0).all(axis=1)]
print(f"모든 조건 모두에서 count가 0인 gene 수: {zero_rna_genes.shape[0]}")

print(df.columns.tolist())



모든 조건 모두에서 count가 0인 gene 수: 18045
['Chr', 'Start', 'End', 'Strand', 'Length', 'CLIP-35L33G.bam', 'RNA-control.bam', 'RNA-siLin28a.bam', 'RNA-siLuc.bam', 'RPF-siLin28a.bam', 'RPF-siLuc.bam', 'RNA_change', 'binding_region']


COUNT가 0인 Gene에서 binding region이 CDS, UTR을 제외한 곳일 가능성이 높음. YOA3에서 확인해볼 것. 